In [20]:
import altair as alt
import pandas as pd

In [21]:
# Load your data
data = pd.read_csv("/Users/brianmetz/Desktop/SI 649/Final Project/MonthlyAbortionProvisionMonthly.csv")  # Replace with your dataset path

# Convert the 'month' column to datetime
data['month'] = pd.to_datetime(data['month'])

In [23]:
# Define states with no data
no_data_states = [
    'AL', 'AR', 'ID', 'KY', 'LA', 'MS',
    'MO', 'ND', 'OK', 'SD', 'TN',
    'TX', 'WV'
]

# Add a note for states with no data
data['note'] = data['state'].apply(
    lambda x: 'No data available for this state due to a total abortion ban.' if x in no_data_states else ''
)

# Include all states in the dropdown, including those with no data
all_states = sorted(data['state'].unique().tolist() + no_data_states)
all_states = ['United States'] + [state for state in all_states if state != 'United States']

# Replace "US" with "United States"
data['state'] = data['state'].replace('US', 'United States')

# Precompute the uncertainty interval as a formatted string
data['uncertainty_interval'] = (
    data['lowerbound'].round(0).astype(int).astype(str) + "–" + data['upperbound'].round(0).astype(int).astype(str)
)

# Adjust the month column to align with the first day of each month
data['month'] = pd.to_datetime(data['month']).dt.to_period('M').dt.to_timestamp()

# Create a dropdown selector with "United States" as the default value
state_dropdown = alt.binding_select(options=all_states, name='Select State')
state_selection = alt.selection_point(fields=['state'], bind=state_dropdown, value='United States')

# Define the hover selection
hover = alt.selection_point(
    fields=['month'],
    nearest=True,
    on='mouseover',
    empty='none',
    clear='mouseout'
)

# Base chart for states with data
base_chart = alt.Chart(data).transform_filter(
    state_selection  # Filter based on selected state
).mark_point(filled=True, size=25, color='purple').encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y')),
    y=alt.Y('median:Q', title='Number of Abortions'),
    tooltip=[
        alt.Tooltip('state:N', title='State'),
        alt.Tooltip('month:T', title='Month', format='%b %Y'),
        alt.Tooltip('median:Q', title='Median estimate of abortions provided', format=','),
        alt.Tooltip('uncertainty_interval:N', title='90% uncertainty interval')
    ]
).add_params(
    hover
)

# Add error bars for uncertainty intervals
error_bars = alt.Chart(data).transform_filter(
    state_selection  # Filter based on selected state
).mark_errorbar(extent='ci', color='purple').encode(
    x='month:T',
    y='lowerbound:Q',
    y2='upperbound:Q'
)

# Add a vertical line for hover interaction
hover_line = alt.Chart(data).mark_rule(color='gray').encode(
    x='month:T'
).transform_filter(
    hover  # Show the line only when hovering
)

# Add a transparent point to capture hover interactions and show the tooltip
hover_points = alt.Chart(data).mark_circle(size=25, color='purple', opacity=1).encode(
    x='month:T',
    y='median:Q',
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%b %Y'),
        alt.Tooltip('median:Q', title='Median estimate of abortions provided', format=','),
        alt.Tooltip('uncertainty_interval:N', title='90% uncertainty interval')
    ]
).add_params(
    hover
).transform_filter(
    state_selection  # Filter based on selected state
)

# Chart for states with no data
no_data_note = alt.Chart(data).transform_filter(
    state_selection  # Filter based on selected state
).transform_filter(
    alt.datum.note != ''  # Only include states with no data
).mark_text(
    align='center',
    fontSize=15,
    color='darkred'
).encode(
    text='note:N'
)

# Combine the charts
final_chart = alt.layer(
    base_chart, error_bars, hover_points, hover_line, no_data_note
).add_params(
    state_selection
).properties(
    title='Detailed Estimates of Clinician-Provided Abortions by State',
    width=700,
    height=400
)

final_chart

alt.LayerChart(...)

In [25]:
# Export each interactive chart to HTML individually
final_chart.save('state_abortion_confidence_intervals.html')